### 0. checking access for GPU

In [ ]:
!nvidia-smi

Mon Sep  4 07:53:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Object_Detection/Licence_Plate_Recognition

/content/drive/MyDrive/Object_Detection/Licence_Plate_Recognition


In [3]:
# Check Dir Root
import os
Home = os.getcwd()
print(Home)

/content/drive/MyDrive/Object_Detection/Licence_Plate_Recognition


### 1. Install YOLOv8 from pip

In [4]:
!pip install ultralytics

from IPython import display
display.clear_output()

In [5]:
# Check ultralystics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.175 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.2/78.2 GB disk)


### 3. Getting custom Dataset

In [6]:
!pip install roboflow
display.clear_output()

In [7]:
!mkdir {Home}/DataSets
%cd {Home}/DataSets

/content/drive/MyDrive/Object_Detection/Licence_Plate_Recognition/DataSets


In [8]:
from roboflow import Roboflow
rf = Roboflow(api_key="3xqoQhgk2T2SnBmvFHiE")
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
dataset = project.version(4).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.175, to fix: `pip install ultralytics==8.0.134`



Extracting Dataset Version Zip to License-Plate-Recognition-4 in yolov8:: 100%|██████████| 48488/48488 [08:21<00:00, 96.75it/s] 


### 4. Custom Training

In [ ]:
%cd {Home}
from ultralytics import YOLO
# Load a model
model = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)

data_path= '/content/drive/MyDrive/Object_Detection/Licence_Plate_Recognition/DataSets/License-Plate-Recognition-4/data.yaml'
# Train the model
result = model.train(data= data_path , epochs=25, imgsz=600,plots=True)

/content/drive/MyDrive/Object_Detection/Licence_Plate_Recognition


Ultralytics YOLOv8.0.175 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/Object_Detection/Licence_Plate_Recognition/DataSets/License-Plate-Recognition-4/data.yaml, epochs=25, patience=50, batch=16, imgsz=600, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False

### 5. Validate Custom Model

In [3]:
# # Validate the model
# metrics = model.val()
# metrics.box.maps   # a list contains map50-95 of each category

### 6. Model Accuracy

In [ ]:
# Train Result
import cv2
img =cv2.imread("/content/runs/detect/train2/results.png")
cv2_imshow(cv2.resize(img, (1000, 600)))

In [ ]:
# 1. Val Confusion Matrix
import cv2
img =cv2.imread("/content/runs/detect/val/confusion_matrix_normalized.png")
cv2_imshow(cv2.resize(img, (800, 600)))

In [ ]:
# 2. Val Precision Curve
img = cv2.imread("/content/runs/detect/val/P_curve.png")
cv2_imshow(cv2.resize(img,(800,600)))

In [ ]:
# 3. Val Recall Curve
img = cv2.imread("/content/runs/detect/val/R_curve.png")
cv2_imshow(cv2.resize(img,(800,600)))

### 6. Inference with Custom Model

In [ ]:
results = model.predict("/content/DataSets/Match-2/test/images", save=True, imgsz=320, conf=0.5)

In [ ]:
# Showing Some predicting results
import glob
for image_path in glob.glob(f'{Home}/runs/detect/predict/*.jpg')[:3]:
      img = cv2.imread(image_path)
      cv2_imshow(cv2.resize(img,(400,400)))
      print("\n")

In [ ]:
img = cv2.imread("/content/images.jpeg")
res = model.predict(img)
res_plotted = res[0].plot()
cv2_imshow(res_plotted)

In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Open the video file
video_path = "/content/1.mp4"
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2_imshow(nce", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()
